In [11]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

^C
Note: you may need to restart the kernel to use updated packages.


In [56]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [2]:
torch.__version__

'2.6.0+cpu'

In [3]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

c:\Users\Marcell Suyanto\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


c:\Users\Marcell Suyanto\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Marcell Suyanto\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [99]:
site = str(input("Site URL: "))
page = requests.get(site)
soup = BeautifulSoup(page.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [100]:
df = pd.DataFrame(np.array(reviews), columns=['reviews'])

In [101]:
df.head()

,reviews
0,Kam Wah is a staple of HK. You must visit if y...
1,Part of the city since 1973 Kam Wah Cafe is he...
2,Pretty tasty meal for breakfast. We came on a ...
3,PSA allergy alert: HK style French toast has p...
4,the best bolo bread ive ever had. you must com...


In [102]:
def sentiment(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)  
    return int(torch.argmax(result.logits)) + 1

In [103]:
df['sentiment'] = df['reviews'].apply(lambda x: sentiment(x))
df['reviews']

0    Kam Wah is a staple of HK. You must visit if y...
1    Part of the city since 1973 Kam Wah Cafe is he...
2    Pretty tasty meal for breakfast. We came on a ...
3    PSA allergy alert: HK style French toast has p...
4    the best bolo bread ive ever had. you must com...
5    While exploring Mong Kok, I stumbled upon this...
6    Wowwwww  our fav fav place haha. I know this m...
7    Must try their pineapple bun! The crust was pe...
8    I ordered the fried rice, french toast, pineap...
9    Came here specifically to try the pineapple bu...
Name: reviews, dtype: object

In [104]:
df

,reviews,sentiment
0,Kam Wah is a staple of HK. You must visit if y...,5
1,Part of the city since 1973 Kam Wah Cafe is he...,5
2,Pretty tasty meal for breakfast. We came on a ...,4
3,PSA allergy alert: HK style French toast has p...,4
4,the best bolo bread ive ever had. you must com...,5
5,"While exploring Mong Kok, I stumbled upon this...",4
6,Wowwwww our fav fav place haha. I know this m...,5
7,Must try their pineapple bun! The crust was pe...,4
8,"I ordered the fried rice, french toast, pineap...",3
9,Came here specifically to try the pineapple bu...,5


In [105]:
mean_sentiment = df['sentiment'].mean()
highest_sentiment = df['sentiment'].max()
lowest_sentiment = df['sentiment'].min()
median_sentiment = df['sentiment'].median()

print(f"Mean: {mean_sentiment}")
print(f"Highest Sentiment: {highest_sentiment}")
print(f"Lowest Sentiment: {lowest_sentiment}")
print(f"Median: {median_sentiment}")
print(f"Number of reviews: {len(df)}")

Mean: 4.4
Highest Sentiment: 5
Lowest Sentiment: 3
Median: 4.5
Number of reviews: 10
